In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import json
import torch
import sys
# sys.path.append('/kaggle/input/bird-clef-utils')
from utils import EffNetB0Classifier, process_spectrogram
torch.manual_seed(42)

device = "cpu"

label_map = json.load(open('label_map.json'))
# Class labels from train audio
class_labels = sorted(list(label_map.keys()))

# List of test soundscapes (only visible during submission)
test_soundscape_path = '/kaggle/input/birdclef-2025/train_soundscapes'
# test_soundscape_path = 'train_soundscapes'
test_soundscapes = [os.path.join(test_soundscape_path, afile) for afile in sorted(os.listdir(test_soundscape_path)) if afile.endswith('.ogg')]

# Open each soundscape and make predictions for 5-second segments
# Use pandas df with 'row_id' plus class labels as columns
predictions = pd.DataFrame(columns=['row_id'] + class_labels)



# Example usage
model = EffNetB0Classifier(num_classes=206)
model.load_state_dict(torch.load("/kaggle/input/first-try/pytorch/default/7/my_model.pth", map_location=torch.device('cpu'), weights_only=True))
model.eval()  # Set the model to evaluation mode
for soundscape in test_soundscapes:
    # Load audio
    sig, rate = librosa.load(path=soundscape, sr=None)

    # Split into 5-second chunks
    chunks = []
    for i in range(0, len(sig), rate * 5):
        chunk = sig[i:i + rate * 5]
        if len(chunk) < rate * 5:  # Pad the chunk if it's less than 5 seconds
            chunk = np.pad(chunk, (0, rate * 5 - len(chunk)), mode='constant')
        chunks.append(chunk)

    # Make predictions for each chunk
    for i, chunk in enumerate(chunks):
        # Compute the spectrogram
        spectrogram = librosa.feature.melspectrogram(y=chunk, sr=rate, n_mels=128)
        spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
        spectrogram_tensor = process_spectrogram(spectrogram_db)
        
        # Get row id (soundscape id + end time of 5s chunk)
        row_id = os.path.basename(soundscape).split('.')[0] + f'_{i * 5 + 5}'

        # Make prediction
        with torch.no_grad():
            logits = model(spectrogram_tensor.unsqueeze(0))
            predictions_array = (logits > 0).int().squeeze().cpu().numpy()

        # Append to predictions as new row
        new_row = pd.DataFrame([[row_id] + list(predictions_array)], columns=['row_id'] + class_labels)
        predictions = pd.concat([predictions, new_row], axis=0, ignore_index=True)

# Save prediction as csv
predictions.to_csv('submission.csv', index=False)
print(predictions)